# Scraping Trustpilot

## Imports

### Packages

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import re
import requests
import time

from bs4 import BeautifulSoup
from datetime import datetime
from functools import partial
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
from wordcloud import WordCloud


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

## Scraping

### Variables globales

In [ ]:
url_trustpilot = 'https://fr.trustpilot.com/review/'

In [ ]:
headers = {
    'User-Agent': 'Antoine Palazzolo - INSEE - Collect data to organize a code exercise',
    'From': 'antoine.palazzolo@insee.fr'
}

In [ ]:
target_companies = ['citizenplane.com',
                    'aeroports-voyages.fr',
                    'www.bourse-des-vols.com',
                    'www.fr.lastminute.com',
                    'ulysse.com',
                    'budgetair.fr',
                    'momondo.fr',
                    'www.promovols.com',
                    'travelgenio.fr',
                    'www.airfrance.fr',
                    'www.ryanair.com',
                    'www.lufthansa.com',
                    'www.easyjet.com',
                    'www.vueling.com',
                    'www.britishairways.com']

In [ ]:
do_scraping = True  # True pour refaire tourner le scraping, False pour importer la dernière version depuis s3

### Fonctions

#### Scraping one review

In [ ]:
def get_note_review(review):
    
    try:
        texte_note = review.find(
            'div', {'class':'star-rating_starRating__4rrcf star-rating_medium__iN6Ty'}
        ).find('img', alt=True)['alt']
        note = int(texte_note[5])  # texte_note = "Noté x étoiles sur 5"
        return note

    except:
        return ""  # "Not found"

In [ ]:
def get_time_review(review):
    
    try:
        str_date = review.find(
            'div', {'class':'typography_body-m__xgxZ_ typography_appearance-subtle__8_H2l styles_datesWrapper__RCEKH'}
        ).find('time')['datetime']
        date = datetime.strptime(str_date, "%Y-%m-%dT%H:%M:%S.%fZ")  # str_date = 'YYYY-MM-DDThh:mm:ss.000Z'
        return date

    except:
        return ""  # "Not found"

In [ ]:
def get_title_review(review):
    
    try:
        title = review.find(
            'h2', {'class':'typography_heading-s__f7029 typography_appearance-default__AAY17'}
        ).text
        return title

    except:
        return ""  # "Not found"

In [ ]:
def get_comment_review(review):
    
    try:
        text = review.find(
            'p', {'class':'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'}
        ).text # Les balises <br> sont supprimées, on va donc forcer des espaces après les points
        text = re.sub(' +', ' ', text.replace('.', '. '))
        return text

    except:
        return ""  # "Not found"

#### Scraping one company

In [ ]:
def get_nb_pages_reviews(url_company):

    time.sleep(5)
    request_text = requests.get(url_company, headers=headers).text
    soup = BeautifulSoup(request_text, 'html.parser')

    try:
        boutons_pages = soup.find(
            'div', {'class':'styles_pagination__6VmQv'}
        ).find_all(
            'span', {'class':'typography_heading-xxs__QKBS8 typography_appearance-inherit__D7XqR typography_disableResponsiveSizing__OuNP7'}
        )
        last_page = int(boutons_pages[-2].text)  # Dernier bouton = "Page Suivante"
        return last_page

    except:
        return 0

In [ ]:
def scraping_one_company(company, limit_pages=False):

    print(company)
    url_company = url_trustpilot+company
    nb_pages = get_nb_pages_reviews(url_company)
    if limit_pages and nb_pages > 2:
        nb_pages = 2
    notes, times, titles, comments = [], [], [], []

    for page in tqdm(range(1, nb_pages+1)):

        time.sleep(5)
        url_page = url_company + '?page=' + str(page)
        
        try:
            request_text = requests.get(url_page, headers=headers).text
            soup = BeautifulSoup(request_text, 'html.parser')
            reviews = soup.find_all(
                'div', {'class':'styles_cardWrapper__LcCPA styles_show__HUXRb styles_reviewCard__9HxJJ'}
            )
        except:
            reviews = []

        notes.extend(list(map(get_note_review, reviews)))
        times.extend(list(map(get_time_review, reviews)))
        titles.extend(list(map(get_title_review, reviews)))
        comments.extend(list(map(get_comment_review, reviews)))

    df_reviews = pd.DataFrame({
        'note': notes,
        'date': times,
        'title': titles,
        'comment': comments
    })

    return df_reviews


#### Scraping all companies

In [ ]:
def scraping_all_companies(companies_list=target_companies,
                           limit_pages=False):

    reviews_companies = list(map(
        partial(scraping_one_company, limit_pages=limit_pages),
                companies_list
    ))
    n = len(companies_list)
    for i in range(n):
        reviews_companies[i]['company'] = companies_list[i]
    df_all_reviews = pd.concat(reviews_companies)
    
    return df_all_reviews

### Mise en pratique

In [ ]:
if do_scraping:
    df_all_reviews = scraping_all_companies(
        limit_pages=False  # limit_pages=False
    )
else:
    os.system(f"mc cp s3/projet-funathon/2024/sujet4/diffusion/reviews_planes.parquet reviews_planes.parquet")
    df_all_reviews = pd.read_parquet("reviews_planes.parquet")

In [ ]:
print(f'{len(df_all_reviews)} avis réunis')

df_all_reviews.sample(5)

In [ ]:
df_problemes = df_all_reviews[df_all_reviews.apply(lambda row: row.str.contains('Not found').any(), axis=1)]
df_problemes

In [ ]:
df_all_reviews = df_all_reviews[df_all_reviews["comment"] != "Not found"]

In [ ]:
df_all_reviews.to_parquet("reviews_planes.parquet", index=False)

In [ ]:
os.system(f"mc cp reviews_planes.parquet s3/projet-funathon/2024/sujet4/diffusion/reviews_planes.parquet")

## Utilisation des avis

### Regarder les notes

In [ ]:
plt.hist(df_all_reviews['note'], bins=5)

In [ ]:
df_all_reviews_by_company = df_all_reviews.groupby('company')['note'].apply(list)

# Créer une grille de sous-graphiques
n_companies = len(df_all_reviews_by_company)
fig, axes = plt.subplots(nrows=1, ncols=n_companies, figsize=(12, 5))

# Parcourir les données et créer un histogramme pour chaque entreprise sur un sous-graphique distinct
for i, (company, notes) in enumerate(df_all_reviews_by_company.items()):
    ax = axes[i]  # Sélectionner le sous-graphique correspondant
    ax.hist(notes, bins=5)  # Créer l'histogramme
    ax.set_xlabel('Notes')  # Définir l'étiquette de l'axe x pour le sous-graphique
    ax.set_ylabel('Fréquence')  # Définir l'étiquette de l'axe y pour le sous-graphique
    ax.set_title(company)  # Définir le titre du sous-graphique

# Ajuster les espacements entre les sous-graphiques
plt.tight_layout()

# Afficher le graphique
plt.show()

### Regarder les commentaires

#### Longueurs des commentaires

In [ ]:
df_all_reviews['len_title'] = df_all_reviews['title'].str.len()
df_all_reviews['len_comment'] = df_all_reviews['comment'].str.len()

In [ ]:
plt.hist(df_all_reviews['len_title'], bins=20)

In [ ]:
plt.hist(df_all_reviews['len_comment'], bins=20)

In [ ]:
plt.hist(df_all_reviews['len_comment'], bins=30, range=(0,1500))

#### Nuage de mots

In [ ]:
stopwords_fr = set(stopwords.words('french'))

# Ajouter les mots "not" et "found" à la liste des stopwords
# stopwords_fr.update(["not", "found"])

# Ajouter manuellement les observations inintéressantes
stopwords_fr.update(["c'est", "j'ai", "donc", "tout", "très"])

In [ ]:
# Concaténer tous les commentaires en une seule chaîne de caractères
all_comments = ' '.join(df_all_reviews['comment'])

# Tokenization des commentaires en mots individuels
tokens = word_tokenize(all_comments)

# Filtrer les mots avec les stopwords en français
filtered_tokens = [word.lower() for word in tokens if word.lower() not in stopwords_fr]

In [ ]:
# Créer un nuage de mots avec les mots filtrés
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(filtered_tokens))

In [ ]:
# Afficher le nuage de mots
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Créer un nuage de mots pour chaque valeur de 'company'
for company in target_companies:
    # Filtrer les commentaires pour la valeur de 'company' spécifique
    comments = df_all_reviews[df_all_reviews['company'] == company]['comment']
    
    # Concaténer tous les commentaires en une seule chaîne de caractères
    all_comments = ' '.join(comments)

    # Tokenization des commentaires en mots individuels
    tokens = word_tokenize(all_comments)

    # Filtrer les mots avec les stopwords en français
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stopwords_fr]

    # Créer un nuage de mots avec les mots filtrés
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(filtered_tokens))

    # Afficher le nuage de mots avec le nom de la société
    plt.figure(figsize=(6, 3))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(company)
    plt.axis('off')
    plt.show()